# Financial Report

In [1]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [2]:
import autogen

llm_config = {
    "model": "gpt-4o-mini", 
    "api_key": os.environ["OPENAI_API_KEY"]
    }

## App 목표

* Asset ticker 또는 회사명을 입력으로 받습니다.
* Asset 자산 가격 데이터를 다운로드하여 분석합니다(상관관계, 수익률 등)
* P/E ratio, ROE 등의 지표를 검색하거나 계산합니다.
* 주식 간 상관관계를 분석합니다.  
* 비교를 위해 정규화된 가격을 표시합니다.  
* 이러한 자산에 대한 최신 뉴스 헤드라인을 다운로드한 다음 이러한 뉴스 헤드라인을 분석하고 요약합니다.  
* 이 모든 정보를 기반으로 이러한 자산에 대한 재무 보고서를 작성합니다.  
    * **Legal reviewer**: 콘텐츠가 법률을 준수하는지 확인합니다.
    * **Text/Data alignment reviewer**: 텍스트 콘텐츠가 보고서에 작성된 텍스트와 일치하는지 확인합니다.
    * **Consistency reviewer**: 텍스트 콘텐츠가 보고서에 작성된 텍스트와 일치하는지 확인합니다.
    * **Completion reviewer**: 보고서에 요청된 모든 요소가 포함되어 있는지 확인합니다. 
* 리뷰에 따라 보고서를 정제합니다.
* 최종 보고서를 마크다운 파일로 저장합니다.


## Agents definition

앱은 에이전트에 의존하여 목표를 달성합니다.

### 1. Financial & Research task

첫 번째 작업은 선택한 자산에 대한 재무 데이터를 얻고 성과 지표 중 일부를 계산하는 것이고, 두 번째 작업은 뉴스 헤드라인을 기반으로 이 성과의 이유를 조사하는 것입니다. 먼저 이러한 작업에 대한 프롬프트를 제공하겠습니다.

In [3]:
from datetime import datetime

date_str = datetime.now().strftime("%Y-%m-%d")

stocks = "네이버, 카카오"

financial_tasks = [
    f"""오늘 날짜는 {date_str} 입니다. 
    {stocks}의 현재 주가는 얼마이고, 지난 6개월 동안의 성과는 백분율 변화 측면에서 어떻습니까?
    각 주식의 전체 이름을 검색하여 모든 향후 요청에 사용합니다.
    이러한 주식의 정규화된 가격 수치를 준비하여 normalized_prices.png라는 파일에 저장합니다. 해당되는 경우 정보를 포함합니다.: 
    * P/E ratio
    * Forward P/E
    * Dividends
    * Price to book
    * Debt/Eq
    * ROE
    * 주식 간의 상관관계를 분석합니다.
    API 키가 필요한 솔루션을 사용하지 마세요.
   일부 데이터가 의미가 없는 경우(예: 가격 0) 쿼리를 변경하고 다시 시도하세요.""",

    """네이버 뉴스 또는 구글 검색에서 시장 뉴스 헤드라인을 활용하여 주식 성과의 가능한 이유를 조사합니다. 파이썬을 사용하여 뉴스 헤드라인을 검색하여 반환합니다. 주식 전체 이름를 사용하여 헤드라인을 검색합니다. 주식당 최소 10개의 헤드라인을 검색합니다. API 키가 필요한 솔루션을 사용하지 마십시오.""",
]

## 작업을 수행할 에이전트 정의

In [4]:
financial_assistant = autogen.AssistantAgent(
    name="Financial_assistant",
    llm_config=llm_config,
)
research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
)

At the end of this task, we will have the price data, performance metrics and a chart that shows normalized price evolution fot the specified assets. 

### 2. Writing task

The second taks will be to write a first version of a report based on the data provided by the previous agents. Let's define the task:

In [5]:
writing_tasks = [
        """제공된 모든 정보를 사용하여 매력적인 재무 보고서를 개발하고 normalized_prices.png 수치와
           제공된 경우 다른 수치를 포함합니다.
           제공된 정보에 주로 의존합니다.
           모든 기본 비율과 데이터를 비교하는 표를 만듭니다.
           모든 기본 비율과 데이터에 대한 의견과 설명을 제공합니다.
           주식을 비교하고 상관 관계와 위험을 고려하고 주식에 대한 비교 분석을 제공합니다.
           각 주식에 대한 최근 뉴스를 요약합니다.
           각 주식에 대한 뉴스 헤드라인에 대한 의견을 제공하고 요약하고 뉴스에 대한 포괄적인 분석을 제공합니다.
           제공된 뉴스 헤드라인과 기본 비율 간의 연결을 제공합니다.
           가능한 미래 시나리오에 대한 분석을 제공합니다.
           그리고, 한글로 작성합니다.
        """]

Let's define an agent to work on this task:

In [6]:
writer = autogen.AssistantAgent(
    name="writer",
    llm_config=llm_config,
    system_message="""
        당신은 전문 작가로, 통찰력 있고 매력적인 재무 보고서로 유명합니다.
        복잡한 개념을 설득력 있는 내러티브로 변환합니다.
        분석에 맥락으로 제공된 모든 지표를 포함합니다.
        마크다운으로 직접 작성된 재무 보고서로만 답변하고 마크다운 언어 블록 표시기를 포함하지 마십시오.
        추가 설명 없이 최종 작업만 반환합니다. 반환 내용은 한글로 작성합니다.
        """,
)

### 3. 블로그 게시물 정제
SEO, 법적 측면 및 윤리적 측면을 기반으로 블로그 게시물을 정제하기 위해 작성자가 Critic에게 연락하면 트리거되는 중첩 채팅을 사용할 것입니다.  
Critic은 각 측면을 최적화하도록 설계된 에이전트와 중첩 채팅을 트리거합니다.    
이 중첩 채팅의 마지막 에이전트인 Meta Reviewer는 모든 리뷰를 요약하고 모아 Critic에게 다시 보내고,   
Critic은 이를 Writer 에이전트에게 전송하여 정제된 버전을 준비합니다. 


In [7]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="당신은 비평가입니다. "
        "당신은 작가의 작업을 검토하고 건설적인 피드백을 제공하여 콘텐츠의 품질을 개선합니다."
        "콘텐츠의 품질을 개선을 한글로 합니다.",
)

legal_reviewer = autogen.AssistantAgent(
    name="LegalReviewer",
    llm_config=llm_config,
    system_message="당신은 법률 검토자입니다. "
        "콘텐츠가 법적으로 준수되고 잠재적인 법적 문제가 없도록 보장하는 능력으로 유명합니다. "
        "제안이 간결하고(3가지 요점 이내), 구체적이고 요점을 잘 잡았는지 확인하세요. "
        "당신의 역할을 명시하여 리뷰를 시작하세요. 리뷰는 한글로 작성합니다.",
)

consistency_reviewer = autogen.AssistantAgent(
    name="ConsistencyReviewer",
    llm_config=llm_config,
    system_message="당신은 일관성 검토자입니다. "
        "보고서 전체에 걸쳐 내용이 일관성을 유지하도록 하세요. "
        "모순이 있는 경우 어떤 버전을 선택해야 하는지 결정하기 위해 보고서의 숫자와 데이터를 참조하세요. "
        "제안이 간결하고(3가지 요점 이내), 구체적이고 요점을 잘 잡았는지 확인하십시오. "
        "역할을 명시하여 검토를 시작하십시오. 검토 내용은 한글로 작성합니다. ",
)

textalignment_reviewer = autogen.AssistantAgent(
    name="TextAlignmentReviewer",
    llm_config=llm_config,
    system_message="당신은 텍스트 데이터 정렬 검토자로, "
        "작성된 콘텐츠의 의미가 텍스트에 쓰여진 숫자와 일치하도록 하는 능력이 있습니다. "
        "텍스트에서 제공된 숫자를 모순 없이 명확하게 설명해야 합니다. "
        "제안이 간결하고(3가지 요점 이내), 구체적이고 요점을 잘 잡았는지 확인하세요. "
        "당신의 역할을 명시하여 검토를 시작하세요. 검토 내용은 한글로 작성합니다. ",
)

completion_reviewer = autogen.AssistantAgent(
    name="CompletionReviewer",
    llm_config=llm_config,
    system_message="당신은 콘텐츠 완성 검토자로, "
        "재무 보고서에 필요한 모든 요소가 포함되어 있는지 확인하는 능력으로 유명합니다. "
        "귀하는 항상 보고서에 다음이 포함되어 있는지 확인합니다. "
        "각 자산에 대한 뉴스 기사, "
        "다양한 비율과 가격에 대한 설명, "
        "가능한 미래 시나리오에 대한 설명, 기본 비율을 비교하는 표, "
        "최소한 하나의 수치."
        "제안이 간결하고(3가지 요점 이내), 구체적이고 요점을 잘 잡았는지 확인하세요."
        "당신의 역할을 명시하여 검토를 시작합니다. 검토 내용은 한글로 작성합니다. ",
)

meta_reviewer = autogen.AssistantAgent(
    name="MetaReviewer",
    llm_config=llm_config,
    system_message="귀하는 메타 검토자로, 다른 검토자의 작업을 집계하고 검토하고 "
        "콘텐츠에 대한 최종 제안을 합니다. 제안 내용은 한글로 작성합니다. ",
)

### 4. 마크다운으로 블로그 게시물 내보내기

마크다운 파일로 최종 블로그 게시물을 내보내는 데 필요한 파이썬 코드를 작성하는 마지막 에이전트를 정의합니다.

In [8]:
exporting_task = ["""python script 를 사용하여 블로그 게시물을 .md 파일로 저장합니다."""]

In [9]:
export_assistant = autogen.AssistantAgent(
    name="Exporter",
    llm_config=llm_config,
)

### 코드 실행

코드를 실행하기 위한 에이전트가 필요합니다.  
입력을 요청하지 않는 사용자 프록시 에이전트를 만들 것입니다.   

In [9]:
user_proxy_auto = autogen.UserProxyAgent(
    name="User_Proxy_Auto",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },  
)

## Agentic flow

에이전트가 상호작용하는 채팅 flow를 정의할 것입니다. 두 가지 채팅을 정의해야 합니다.
1. 블로그 게시물 작성을 위한 중첩 채팅 흐름
2. 주요 채팅 흐름

### Nested chat flow

먼저 중첩된 채팅 flow를 정의하는 것으로 시작하겠습니다.  
이 채팅은 작성자가 비평가에게 연락할 때 트리거됩니다.

In [10]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "리뷰를 한글로 작성합니다. 리뷰를 JSON 객체로만 반환합니다:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": textalignment_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "리뷰를 한글로 작성합니다. 리뷰를 JSON 객체로만 반환합니다:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
    {"recipient": consistency_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "리뷰를 한글로 작성합니다. 리뷰를 JSON 객체로만 반환합니다:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
    {"recipient": completion_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "리뷰를 한글로 작성합니다. 리뷰를 JSON 객체로만 반환합니다:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "모든 reviewer의 피드백을 집계하고 글쓰기에 대한 최종 제안을 제공합니다. 최종 제안을 한글로 작성합니다.", 
     "max_turns": 1},
]

critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

### Main chat flow

이제 주요 채팅 흐름을 정의해 보겠습니다. 4개의 채팅으로 구성됩니다.:
1. 첫 번째 재무 작업을 수행하기 위한 사용자 프록시가 있는 재무 에이전트
2. 두 번째 재무 작업을 수행하기 위한 사용자 프록시가 있는 리서치 에이전트
3. 중첩된 채팅을 통해 블로그 게시물을 작성하고 다듬기 위한 작성자가 있는 비평가
4. 마크다운 블로그 게시물을 파일로 내보내기 위한 사용자 프록시가 있는 내보내기 에이전트

In [11]:
chat_results = autogen.initiate_chats(
    [
        {
            "sender": user_proxy_auto,
            "recipient": financial_assistant,
            "message": financial_tasks[0],
            "silent": False,
            "summary_method": "reflection_with_llm",
            "summary_args": {
                "summary_prompt" : "주식의 주가, 실적 및 기타 모든 지표를 JSON 객체로만 반환합니다."
                "생성된 모든 그림 파일의 이름을 제공합니다. 각 주식의 전체 이름을 제공합니다.",
            },
            "clear_history": False,
            "carryover": "대화를 종료하기 전에 코드 실행 확인을 기다리세요. 데이터가 NaN 값으로 완전히 구성되지 않았는지 확인하세요. 모든 것이 완료되면 마지막에 TERMINATE로 응답하세요."
        },
        {
            "sender": user_proxy_auto,
            "recipient": research_assistant,
            "message": financial_tasks[1],
            "silent": False,
            "summary_method": "reflection_with_llm",
            "summary_args": {
                "summary_prompt" : "각 주식에 대한 뉴스 헤드라인을 문단으로 제공하고, 정확해야 하지만 모호한 뉴스 이벤트는 고려하지 말고, JSON 객체로만 결과를 반환하세요."
                            },
            "clear_history": False,
            "carryover": "대화를 종료하기 전에 코드 실행 확인을 기다리세요. 데이터가 NaN 값으로 완전히 구성되지 않았는지 확인하세요. 모든 것이 완료되면 마지막에 TERMINATE로 응답하세요."
        },
        {
            "sender": critic,
            "recipient": writer,
            "message": writing_tasks[0],
            "carryover": "제공된 데이터의 수치와 표를 재무 보고서에 포함시키고 싶습니다.",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "sender": user_proxy_auto,
            "recipient": export_assistant,
            "message": exporting_task[0],
            "carryover": "대화를 종료하기 전에 코드 실행 확인을 기다리세요. 데이터가 NaN 값으로 완전히 구성되지 않았는지 확인하세요. 모든 것이 완료되면 마지막에 TERMINATE로 응답하세요.",
        }
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
User_Proxy_Auto (to Financial_assistant):

오늘 날짜는 2025-04-06 입니다. 
    네이버, 카카오의 현재 주가는 얼마이고, 지난 6개월 동안의 성과는 백분율 변화 측면에서 어떻습니까?
    각 주식의 전체 이름을 검색하여 모든 향후 요청에 사용합니다.
    이러한 주식의 정규화된 가격 수치를 준비하여 normalized_prices.png라는 파일에 저장합니다. 해당되는 경우 정보를 포함합니다.: 
    * P/E ratio
    * Forward P/E
    * Dividends
    * Price to book
    * Debt/Eq
    * ROE
    * 주식 간의 상관관계를 분석합니다.
    API 키가 필요한 솔루션을 사용하지 마세요.
   일부 데이터가 의미가 없는 경우(예: 가격 0) 쿼리를 변경하고 다시 시도하세요.
Context: 
대화를 종료하기 전에 코드 실행 확인을 기다리세요. 데이터가 NaN 값으로 완전히 구성되지 않았는지 확인하세요. 모든 것이 완료되면 마지막에 TERMINATE로 응답하세요.

--------------------------------------------------------------------------------
Financial_assistant (to User_Proxy_Auto):

이 작업을 수행하려면 여러 단계로 나누어 진행하겠습니다. 

1. **현재 및 과거 데이터 검색**: 네이버와 카카오의 현재 주가를 알아내고, 지난 6개월 동안의 주가 데이터를 가져오겠습니다. 이 정보는 `yfinance`와 같은 라